# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

**Descriping the model:**

We consider a version of the Ramsey model with population growing at rate $1+n$.


**Households maximize** the intertemporal utility function:

$$
\begin{aligned}
U =& \max_{c_{t}}\sum_{t=0}^{\infty}\beta^{t}(1+n)^{t}\frac{c_{t}^{1-\theta}}{1-\theta}\\
 & \text{s.t.}\\
 & a_{t+1}(1+n)=a_tR_t+w_t-c_t
\end{aligned}
$$
where $c_t$ is consumption per capita, $\beta$ is the discount rate, $\theta$ is the coefficient of relative risk aversion and $a_t$ .
with $a_0>0$ given and $R_t=1+r_t-\delta$ implies that return on household savings ($R_t$) must equal the rental rate on capital paid by firms ($r_t$) plus the undepreciated capital ($1-\delta$)
It is assumed $\beta(1+n)<1$ and $\theta>0$



The Lagrangian of the representative household's optimization problem is:
$$
\begin{equation*}
L=\sum_{t=0}^{\infty}\beta^{t}(1+n)^{t}\left[\frac{c_{t}^{1-\theta}}{1-\theta}-\mu_t\left(a_{t+1}(1+n)-(1+r_t-\delta)a_t-w_t+c_t\right)\right]
\end{equation*}
$$
where $\mu_t=\frac{\lambda_t}{\beta^t(1+n)^t}$

The FOC's are:
$$
\begin{aligned}
\frac{\partial L}{\partial c_t}=0 &\Leftrightarrow \mu_t=c_t^{-\theta} \\
\frac{\partial L}{\partial a_{t+1}}=0 &\Leftrightarrow \beta \mu_{t+1}(1+r_{t+1}-\delta)
\end{aligned}
$$

Which combined yields the Euler equation, describing the households optimal consumption path:
$$
\begin{equation*}
c_t^{-\theta}=\beta c_{t+1}^{-\theta}(1+r_{t+1}-\delta) \Leftrightarrow \frac{c_{t+1} }{c_t} = \left(\beta(1+r_{t+1}-\delta)\right)^{\frac{1}{\theta}}
\end{equation*}
$$
where $\frac{1}{\theta}$ is the elasticity of substitution, which is a measure of how willing the household is to substitute a unit of consumption today for a unit of consumption tomorrow.

**Identical competitive firms maximize** the profit function:

$$
\begin{equation*}
\pi^F(K_t,L_t)=A K_t^{\alpha}(L_t)^{1-\alpha}-w_tL_t-r_tK_t
\end{equation*}
$$

where $L_t$ is labor, $K_t$ is capital, $w_t$ is the wage rate, $r_t$ is the rental rate.

Profit maximization yields:
$$
\begin{aligned}
r_t =& A\alpha k_t^{\alpha-1} \\
w_t =& A(1-\alpha) k_t^{\alpha}
\end{aligned}
$$
where $k_t=\frac{K_t}{L_t}$ is capital per capita.

Furthermore, the goods market clearing condition is:
$$
Y_t=c_t+I_t
$$
where $Y_t=F(K_t,L_t)$

And capital follows the law of motion:
$$
K_t=(1-\delta)K_{t-1}+I_t
$$


**Solution Algorithm:**
  

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 